# Scaling & Hyperparameter Tuning

![](https://i.ytimg.com/vi/b0L_2jKEbA4/maxresdefault.jpg)

In this notebook you will see a short example of how to scale your data and tune the hyperparamters of your models using grid or random search. 

We will use the well-known titanic dataset. Since you've already worked your way through the steps of exploring and cleaning the data as well as selecting proper features for modelling in another notebook, we will skip this part here and use the **preprocessed data** from the logistic regression notebook. 

In [ ]:
# Import packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from timeit import default_timer as timer

# eye candy plots
#plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')
# source https://github.com/dhaitz/matplotlib-stylesheets

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

RSEED = 10

In [ ]:
# Import data 
df = pd.read_csv('data/titanic_preprocessed.csv')
df.head()

In [ ]:
# Check for missing data
df.isnull().sum()

## Train-test-split

We will define the target and predictors and split our dataset into a train and test set.

In [ ]:
# Define predictors and target
y = df.Survived
X = df.drop('Survived', axis=1)

In [ ]:
# Check X
X.head(2)

In [ ]:
# Train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

---
## Feature Scaling

Often the input features of your model have different units which means that the variables also have different scales. While some model types (e.g. tree-based models like decision tree or random forest) are unaffected by the scale of numerical input variables, many machine learning algorithms including for example algorithms using distance measures (e.g. KNN, SVM) perform better when the input features are scaled to a specific range. 

The most popular techniques for scaling are **normalization** and **standardization**. 

Check the [link](https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/) for further info. 

![](images/normalization_vs_standardization.png)

In [ ]:
# Before we have a look at the different methods, 
# we have to define which columns we want to scale.
col_scale = ['Age', 'SibSp', 'Parch', 'Fare']

### Data Standardization 

In order to standardize a dataset it is necessary to rescale the distribution of values so that the mean of observed values is 0 and the standard deviation is 1. You can think of it as subtracting the mean value or centring the data. 
Sklearn provides us for this case with the [Standard scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

A value is standardized as follows: 

$ x_{scaled} = \frac{x – \mu}{\sigma}  $, where 

$ \mu = \frac{\sum{x}}{m} $ is the mean, where m is the number of observations

$ \sigma = \sqrt{ \frac{\sum{ (x – \mu)^2 }}{m}} $ is the standard deviation

In [ ]:
# transformation with column transformer
scaling_transformation = ColumnTransformer([
    ('scaling', StandardScaler(), col_scale)],
    remainder='passthrough'
)

### Data normalization 

Normalizing the data means to rescale it from the original range so that all values lie within the new range of 0 and 1.
We can easily do this by using the [Min-Max-Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) from sklearn. This scaler transforms the feature(s) by scaling it(them) to a given range (default range is 0 to 1). 

A value is normalized as follows: 

$ x_{scaled} = \frac{x – x_{min}}{x_{max} – x_{min}} $

(Where the min and max values pertain to the value x being normalized, from your **train** dataset)

### ex1. Practice round
Try out doing normalizing

In [ ]:
# Scaling with MinMaxScaler

# Try to scale you data with the MinMaxScaler() from sklearn. 
# It follows the same syntax as the StandardScaler.
# Don't forget: you have to import the scaler at the top of your notebook. 

### Bonus Question: Do the test standardized / normalized values follow the same properties of the train standardized / normalized values?

i.e standardized: centered on 0 with stdev of 1 / normalized w values between 0 and 1

---
## Predictive Modelling

We will evaluate our model performance with a quick and more reliable way using sklearn's [cross_val_score()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) which implements K-fold cross validation. When training a model based on train and test split we only have one experiment. Can we really trust one experiment? 

Think of [K-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) as doing K experiments and then taking the average error. It is still not perfect but better than 1 experiment which can randomly turn out to be really good. 

Whenever we have K, comes the question about the value of K.. common values are between 5 and 10 and you need to take into account the technical limitations: dataset size, compute power and available memory and time. CV takes time on large datasets.


![](images/cross_validation.png)

### KNNClassifier - unscaled data

In [ ]:
# Fit and evaluate model without hyperparameter tuning using cross validation and unscaled data 
knn_classifier = KNeighborsClassifier()
scores = cross_val_score(knn_classifier, X_train, y_train, cv=5, n_jobs=-1)

# Evaluation 
print('Score (unscaled):', round(scores.mean(), 2))
print('Standard Deviation (unscaled):', round(scores.std(), 2))

In [ ]:
# plotting the scores and average score
plt.axhline(y=scores.mean(), color='y', linestyle='-')
sns.barplot(x=[1,2,3,4, 5],y=scores).set_title('Scores of the K-Folds Models - unscaled data');

### KNNClassifier - scaled data

In [ ]:
# Fit and evaluate model without hyperparameter tuning using cross validation and scaled data 
knn_classifier_scaled = Pipeline([
    ('scaling_trans', scaling_transformation ),
    ('knn',KNeighborsClassifier())
])
scores_scaled = cross_val_score(knn_classifier_scaled, X_train, y_train, cv=StratifiedKFold(5), n_jobs=-1)

# Evaluation
print('Score (scaled):', round(scores_scaled.mean(), 2))
print('Standard Deviation (unscaled):', round(scores.std(), 2))

In [ ]:
plt.axhline(y=scores_scaled.mean(), color='y', linestyle='-')
sns.barplot(x=[1,2,3,4, 5],y=scores_scaled).set_title('Scores of the K-Folds Models - standardized data');

You can see that the overall error has gone down significantly (in my case an increase of 13% in the evaulation score), but the errors of the folds are rather similar to each. To show the effect of scaling a bit clearer we will look at another classifier that is even more affected by it.  

### SGDClassifier - unscaled data

SGD classifier stands for stochastic gradient descent classifier and implements a variety of classifiers - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html. The model it fits can be controlled with the loss parameter; by default, it fits a linear support vector machine (SVM). We will use it here to further emphasise the importance of scaling and will set the loss parameter to `log_loss`. This way the model will be a logistic regression using stochastic gradient descent for learning.

In [ ]:
# Fit and evaluate model without hyperparameter tuning using cross validation and unscaled data 
sgd_classifier = SGDClassifier(random_state=RSEED, loss ='log_loss')
scores = cross_val_score(sgd_classifier, X_train, y_train, cv=StratifiedKFold(5), n_jobs=-1)

# Evaluation 
print('Score (unscaled):', round(scores.mean(), 2))
print('Standard Deviation (unscaled):', round(scores.std(), 2))

In [ ]:
# plotting the scores and average score
plt.axhline(y=scores.mean(), color='y', linestyle='-')
sns.barplot(x=[1,2,3,4, 5],y=scores).set_title('Scores of the K-Folds Models - unscaled data');

The validation error varies a lot for the different folds of the data (from around 0.5 to a high of almost 0.8)

In [ ]:
# Fit and evaluate model without hyperparameter tuning using cross validation and scaled data 
sgd_classifier_scaled = Pipeline([
    ('scaling_trans', scaling_transformation ),
    ('sgd', SGDClassifier(random_state=RSEED,loss ='log_loss'))

])

scores_scaled = cross_val_score(sgd_classifier_scaled, X_train, y_train, cv=StratifiedKFold(5), n_jobs=-1)

# Evaluation
print('Score (scaled):', round(scores_scaled.mean(), 2))
print('Standard Deviation (unscaled):', round(scores.std(), 2))

In [ ]:
plt.axhline(y=scores_scaled.mean(), color='y', linestyle='-')
sns.barplot(x=[1,2,3,4, 5],y=scores_scaled).set_title('Scores of the SGD Models - standardized data');

Beside the obviously better overall performance an important effect of scaling visible here is that the validation error doesn't vary quite as much as it used with most of the values being closer to the 0.8 mark. 

### ex2. Practice round
Try out KNNClassifier with the normalized data

In [ ]:
# Train model using the MinMaxScaler data you scaled in ex1.

# It follows the same syntax as the StandardScaler.

---
## Hyperparameter Tuning

Most models have many hyperparameters that work better with some datasets than with others. Same goes with the hyperparameters from regularization which we learned that are selected based on a trial and error process. So how do we manage to select the parameter values that work best for our data?

#### GridSearchCV

[Grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) is a tuning technique that attempts to compute the optimum values of hyperparameters. It performs an exhaustive search over a prior defined parameter space using cross-validation (hence the **CV** suffix). That means it will evaluate all of the possible parameter combinations of the search space in order to find and return the best combination. 


This task, however, starts to become very time-consuming if there are many hyperparameters and the search space is huge. As you can see for k= 5 and for 2 parameters with 2, and respectively 3 values, thus 6 combinations, the GridSearchCV runs 30 modelling steps in order to just come up with the best values for the two parameters.

![](images/grid_search_cv.png)

In [ ]:
#what hyperparameters does KNN have?
knn_classifier_scaled.get_params()

In [ ]:
# Defining parameter grid (as dictionary)
param_grid = {"knn__n_neighbors" : [2,4,3,5,10], #this actually defines the model you use
              "knn__weights" : ["uniform", "distance"],
              "knn__p" : [1, 2, 3],
             }

# Instantiate gridsearch and define the metric to optimize 
gs = GridSearchCV(knn_classifier_scaled, param_grid, scoring='accuracy',
                  cv=5, verbose=1, n_jobs=-1)

# Fit gridsearch object to data.. also lets see how long it takes
start = timer()
gs.fit(X_train, y_train)
end = timer()
gs_time = end-start

In [ ]:
gs_time

In [ ]:
# Best score
print('Best score:', round(gs.best_score_, 3))

# Best parameters
print('Best parameters:', gs.best_params_)

In [ ]:
# we will do this at least twice.. according to DRY we should write a function
def print_pretty_summary(name, best_model, y_test, y_pred_test):
    print(name)
    print('=======================')
    print('n_neighbors: {}'.format(best_model.steps[-1][1].n_neighbors))
    print('weights: {}'.format(best_model.steps[-1][1].weights))
    print('p: {}'.format(best_model.steps[-1][1].p))
    print('algorithm: {}'.format(best_model.steps[-1][1].algorithm))

    accuracy = accuracy_score(y_test, y_pred_test)
    print('Test accuracy: {:2f}'.format(accuracy))
    return accuracy

In [ ]:
knn_best = gs.best_estimator_
knn_best

In [ ]:
# Assigning the fitted KNNClassifier model with best parameter combination to a new variable knn_best
knn_best = gs.best_estimator_

# Making predictions on the test set
y_pred_test = knn_best.predict(X_test)
# Let us print out the performance of our model on the test set.
knn_accuracy = print_pretty_summary('KNNClassifier model', knn_best, y_test, y_pred_test)

#### [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

As an alternative to grid search we can use sklearn's RandomizedSearchCV(). Random search will not try every possible combination of our search space but will randomly pick and evaluate parameter combinations. 

In [ ]:
# Define parameter grid for randomized search
param_grid = {"knn__n_neighbors" : [2,4,3,5,10], #this actually defines the model you use
              "knn__weights" : ["uniform", "distance"],
              "knn__p" : [1, 2, 3],
             }
# Instantiate random search and define the metric to optimize 
rs = RandomizedSearchCV(knn_classifier_scaled, param_grid, scoring='accuracy',
                  cv=5, verbose=5, n_jobs=-1, random_state=RSEED)

# Fit randomized search object to data
start = timer()
rs.fit(X_train, y_train)
end = timer()
rgs_time = end-start

In [ ]:
# Best score
print('Best score:', round(rs.best_score_, 3))

# Best parameters
print('Best parameters:', rs.best_params_)

In [ ]:
# Assigning the fitted KNNClassifier model with best parameter combination to a new variable knn_best
knn_best_rs = rs.best_estimator_

# Making predictions on the test set
y_pred_test_rs = knn_best_rs.predict(X_test)


# Let us print out the performance of our model on the test set.
rknn_accuracy = print_pretty_summary('KNNClassifier model (randomizedGSCV)', knn_best_rs, y_test, y_pred_test_rs)

In [ ]:
print(f"Grid search took {gs_time} seconds to run with accuracy: {knn_accuracy:f}")
print(f"Randomized Grid search took {rgs_time} seconds to run with accuracy: {rknn_accuracy:f}")

This is an educational example... more data means longer train times